In [1]:
env = "production"
# env = "staging"


# s3_bucket = "s3://mist-aggregated-stats-{env}/entity_event/entity_event-{env}/".format(env=env)
date_day = "2020-03-1*"
hr = '*'

# s3_capacity_path = "dt={day}/hr={hr}/CapacityAnomalyEvent_*.seq".format(day=date_day, hr=hr)
# s3_coverage_path = "dt={day}/hr={hr}/CoverageAnomalyEvent_*.seq".format(day=date_day, hr=hr)
# s3_capacity_path = s3_bucket + s3_capacity_path
# s3_coverage_path = s3_bucket + s3_coverage_path
# print(s3_capacity_path, "\n", s3_coverage_path)

s3_bucket = "s3://mist-secorapp-staging/sle-capacity-anomaly/sle-capacity-anomaly-staging/".format(env=env)
s3_path = s3_bucket + "dt={day}/hr={hr}/*.seq".format(day=date_day, hr=hr)

print(s3_path)


s3://mist-secorapp-staging/sle-capacity-anomaly/sle-capacity-anomaly-staging/dt=2020-03-1*/hr=*/*.seq


In [2]:
# rdd_capacity = spark.sparkContext.sequenceFile(s3_capacity_path)
# rdd_coverage = spark.sparkContext.sequenceFile(s3_coverage_path)
rdd_capacity = spark.sparkContext.sequenceFile(s3_path)

In [3]:
rdd_capacity.first()

(55555,
 bytearray(b'{"radio_5s":1,"util_nonwifi_std_base":7.062569944280419,"channel":1,"sle_coverage_base":0.9935433190673669,"util_nonwifi_deviation":3.715947279481879,"rssi_mean":-49.70652173913044,"rad_id":"r24","sle_capacity_anomaly_score":0.27366601949014646,"util_all_mean_base":90.93317336113182,"anomaly_score":2.014241404491411,"dev":"r0","anomaly_type":"nonwifi","band":"24","util_cochannel_std":5.0,"sle_coverage_anomaly":false,"util_cochannel_mean":43.43069752097154,"util_cochannel_std_base":5.46560143641446,"events":[],"avg_nclients":1.0,"sle_capacity_error":0.4461205267868592,"interference_anomaly_score":2.014241404491411,"util_nonwifi_mean_base":30.03051711743215,"util_ap":9.792660988750868,"sle_coverage_error":0.4335800950892067,"util_cochannel_deviation":-1.1954569697494748,"ap":"5c-5b-35-0e-cf-eb","rssi_hist":{"-35":62,"-38":62,"-77":60},"error_rate":0.5540877024144466,"sle_coverage":0.6739130434782609,"user_seconds_util_high":184,"sle_coverage_base_std":0.1,"sle_covera

In [4]:
import json
# rdd_site = rdd.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"), x.get("ap_id"))).groupByKey()
# rdd_site = rdd.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"))).groupBy('site_id')
df_capacity = rdd_capacity.map(lambda x: json.loads(x[1])).toDF() #.map(lambda x: json.loads(x[1])).



/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [5]:
# df_capacity
df_capacity.printSchema()

root
 |-- anomaly: string (nullable = true)
 |-- anomaly_score: double (nullable = true)
 |-- anomaly_type: string (nullable = true)
 |-- ap: string (nullable = true)
 |-- avg_nclients: double (nullable = true)
 |-- band: string (nullable = true)
 |-- bandwidth: long (nullable = true)
 |-- channel: long (nullable = true)
 |-- dev: string (nullable = true)
 |-- error_rate: double (nullable = true)
 |-- events: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- impacted_wlans: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- interference_anomaly_score: double (nullable = true)
 |-- interference_type: string (nullable = true)
 |-- interval: long (nullable = true)
 |-- max_nclients: double (nullable = true)
 |-- org: string (nullable = true)
 |-- power: long (nullable = true)
 |-- rad_id: string (nullable = true)
 |-- radio_5s: long (nullable = true)
 |-- rssi_deviation: double (nullable = true)
 |-- rssi_hist: map (nullable = true)
 | 

In [6]:

# df_capacity.select("org", "site",  "anomaly", "anomaly_score", "anomaly_type", "sle_capacity_anomaly", "util_nonwifi_mean", "util_cochannel_mean").show()

df_capacity_g = df_capacity.select("org", "site", "anomaly_type").groupBy("org", "site", "anomaly_type").count()

In [7]:
# df_capacity_g.show()

In [9]:
df_capacity_g.orderBy('count', ascending=False).show(10)

# df_capacity_g.sort('count', ascending=False).show() #


+--------------------+--------------------+------------+-----+
|                 org|                site|anomaly_type|count|
+--------------------+--------------------+------------+-----+
|828ed216-66ad-4d2...|8a1c65c6-1962-477...|   cochannel|   45|
|b4e16c72-d50e-4c0...|57b2f891-09c1-4dc...|   cochannel|   36|
|bb2fb165-0931-49c...|c3b8f61c-c5a2-495...|     nonwifi|   25|
|41f579e0-5a00-416...|c475d117-b152-445...|     nonwifi|   21|
|bb2fb165-0931-49c...|c3b8f61c-c5a2-495...|   cochannel|   15|
|b4e16c72-d50e-4c0...|57b2f891-09c1-4dc...|     nonwifi|   14|
|2dbab92c-7cc4-11e...|6db15e18-bc94-11e...|     nonwifi|   12|
|6748cfa6-4e12-11e...|67970e46-4e12-11e...|     nonwifi|   11|
|1d2e8b2d-6afe-4c1...|713afd5a-6b75-47a...|   cochannel|   10|
|b4e16c72-d50e-4c0...|f688779c-e335-4f8...|   cochannel|   10|
+--------------------+--------------------+------------+-----+
only showing top 10 rows



In [6]:
# # rdd_site.take(1)
# rdd_capacity_by_site= rdd_capacity.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"),1)).countByKey()

# rdd_coverage_by_site= rdd_coverage.map(lambda x: json.loads(x[1])).map(lambda x: (x.get("site_id"),1)).countByKey()

# site_capacity_sorted = sorted(rdd_capacity_by_site.items(), key=lambda v:v[1], reverse=True) #, key=lambda(k, v): v)
# site_coverage_sorted = sorted(rdd_coverage_by_site.items(), key=lambda v:v[1], reverse=True) #, key=lambda(k, v): v)


# site_capacity_sorted[:10]


In [18]:
# site_coverage_sorted[:10]

In [10]:
# rdd_coverage_by_site.sort()